In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash, session,  jsonify, json
from flask_mysqldb import MySQL, MySQLdb
MySQLdb.paramstyle
from werkzeug.utils import secure_filename
import pymysql
import os
import xlrd
import json
import urllib.request
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import mysql.connector
import sys
    
app = Flask(__name__)
app.secret_key = "caircocoders-ednalan"
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = ''
app.config['MYSQL_DB'] = 'toko'
app.config['MYSQL_CURSORCLASS'] = 'DictCursor'
mysql = MySQL(app)

#guest
@app.route("/")
def main():
    return render_template('index.html')

@app.route("/about")
def about():
    return render_template('about.html')

@app.route("/products")
def products():
    conn = mysql.connection.cursor()
    conn.execute('SELECT nama_produk, nama_variasi, CONCAT("Rp. ", FORMAT(harga, 0, "de_DE")) harga, foto FROM produk WHERE `set` = 1')
    produk = conn.fetchall() 
    return render_template('product.html', produk=produk)

#admin
@app.route('/login/', methods=['GET', 'POST'])
def login():
    conn = mysql.connection.cursor()
    msg = ''
    # cek username password
    if request.method == 'POST' and 'username' in request.form and 'password' in request.form:
        username = request.form['username']
        password = request.form['password']
        conn.execute('SELECT * FROM accounts WHERE username = %s AND password = %s', (username, password))
        account = conn.fetchone()
   
        # if cek akun valid
        if account:
            # session data
            session['loggedin'] = True
            session['id'] = account['id']
            session['username'] = account['username']
            return redirect(url_for('home'))
        # akun tidak valid
        else:
            msg = 'Incorrect username/password!'
    return render_template('login.html', msg=msg)

@app.route('/register/', methods=['GET', 'POST'])
def register():
    conn = mysql.connection.cursor()
    msg = ''
    # insert
    if request.method == 'POST':
        if request.form['nama'] != "":
            if request.form['username'] != "": 
                if request.form['email'] != "":
                    if request.form['password'] !="":
                        nama = request.form['nama']
                        username = request.form['username']
                        email = request.form['email']
                        password = request.form['password']
                        conn.execute('SELECT * FROM accounts WHERE username = %s AND password = %s', (username, password))
                        conn.fetchone()
                        cek = conn.rowcount
                        if cek == 0:
                            conn.execute('INSERT INTO `accounts`(`fullname`, `username`, `password`, `email`) VALUES (%s, %s, %s, %s)', (nama, username, password, email))
                            mysql.connection.commit()
                            return redirect(url_for('login'))
                        else:
                            msg = 'Username already used!'
                            return render_template('register.html', msg=msg)
                    else:
                        msg = 'Please insert the information!'
                        return render_template('register.html', msg=msg)
                else:
                    msg = 'Please insert the information!'
                    return render_template('register.html', msg=msg)
            else:
                msg = 'Please insert the information!'
                return render_template('register.html', msg=msg)
        else:
            msg = 'Please insert the information!'
            return render_template('register.html', msg=msg)
    return render_template('register.html', msg=msg)

#dashboard
@app.route('/home')
def home():
    conn = mysql.connection.cursor()
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()
        #total pendapatan
        conn.execute('SELECT CONCAT("Rp. ", FORMAT((SUM(total_harga)), 0, "de_DE")) total FROM penjualan WHERE status_pesanan = "Selesai"')
        total = conn.fetchone()
        #jumlah transaksi
        conn.execute('SELECT COUNT(no_pesanan) transaksi FROM penjualan WHERE status_pesanan = "Selesai"')
        transaksi = conn.fetchone()
        #jumlah pelanggan
        conn.execute('SELECT COUNT(DISTINCT username_pembeli) pelanggan FROM penjualan WHERE status_pesanan = "Selesai"')
        pelanggan = conn.fetchone()
        #5 bulan terakhir
        conn.execute('SELECT DATE_FORMAT(waktu_pesanan_selesai, "%b %y") bulan FROM penjualan WHERE status_pesanan = "Selesai" GROUP BY bulan ORDER BY waktu_pesanan_selesai ASC LIMIT 5 OFFSET 22')
        bulan = conn.fetchall() 
        #total pendapatan/bulan dari 5 bulan terakhir
        conn.execute('SELECT SUM(total_harga) total FROM penjualan WHERE status_pesanan = "Selesai" GROUP BY DATE_FORMAT(waktu_pesanan_selesai, "%b %y") ORDER BY waktu_pesanan_selesai ASC LIMIT 5 OFFSET 22')
        total_bulan = conn.fetchall() 
        #jumlah barang
        conn.execute('SELECT COUNT(DISTINCT nama_produk, nama_variasi) AS barang FROM penjualan')
        barang = conn.fetchone()
        #top 5 pelanggan
        conn.execute('SELECT nama_penerima AS pelanggan, CONCAT("Rp. ", FORMAT((SUM(total_harga)), 0, "de_DE")) AS monetary FROM penjualan WHERE status_pesanan = "Selesai" GROUP BY username_pembeli ORDER BY SUM(total_harga) DESC LIMIT 5')
        top_pelanggan = conn.fetchall()
        #top 5 produk
        conn.execute('SELECT SUBSTR(nama_produk, 1, 50) AS produk, SUM(jumlah) AS jumlah FROM `penjualan` WHERE status_pesanan = "Selesai" AND nama_produk NOT IN ("Pesanan", "TITIPAN") GROUP BY nama_produk ORDER BY jumlah DESC LIMIT 5')
        top_produk = conn.fetchall()
        #jumlah transaksi/bulan dari 5 bulan terakhir
        conn.execute('SELECT COUNT(no_pesanan) transaksi FROM penjualan WHERE status_pesanan = "Selesai" GROUP BY DATE_FORMAT(waktu_pesanan_selesai, "%b %y") ORDER BY waktu_pesanan_selesai ASC LIMIT 5 OFFSET 22')
        total_transaksi = conn.fetchall()
        #top 5 provinsi
        conn.execute('SELECT provinsi, COUNT(no_pesanan) AS jumlah FROM `penjualan` WHERE status_pesanan = "Selesai" GROUP BY provinsi ORDER BY jumlah DESC LIMIT 5')
        propinsi = conn.fetchall()
        
        return render_template('user/index.html', propinsi= propinsi, total_transaksi=total_transaksi, top_produk=top_produk, top_pelanggan=top_pelanggan, account=account, barang=barang, bulan=bulan, total=total, total_bulan=total_bulan, transaksi=transaksi, pelanggan=pelanggan)
    # session tidak ada
    return redirect(url_for('login'))

#convert rupiah
import locale
def rupiah_format(angka, with_prefix=False, desimal=0):
    locale.setlocale(locale.LC_NUMERIC, 'IND')
    rupiah = locale.format("%.*f", (desimal, angka), True)
    if with_prefix:
        return "Rp. {}".format(rupiah)
    return rupiah

#dashboard transaksi
@app.route('/transaksi')
def transaksi():
    conn = mysql.connection.cursor()
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()
        #total pendapatan
        conn.execute('SELECT CONCAT("Rp. ", FORMAT((SUM(total_harga)), 0, "de_DE")) total FROM penjualan WHERE status_pesanan = "Selesai"')
        total = conn.fetchone()
        #jumlah transaksi
        conn.execute('SELECT COUNT(no_pesanan) transaksi FROM penjualan WHERE status_pesanan = "Selesai"')
        transaksi = conn.fetchone()
        
        conn.execute('SELECT SUM(total_harga) total FROM penjualan WHERE status_pesanan = "Selesai"')
        pendapatan = conn.fetchone()
        conn.execute('SELECT COUNT(no_pesanan) AS bulan FROM penjualan WHERE status_pesanan = "Selesai" GROUP BY DATE_FORMAT(waktu_pesanan_selesai, "%b %y")')
        conn.fetchall()
        bulan = conn.rowcount
        #pendapatan/bulan
        a = int(pendapatan['total']) / int(bulan)
        a = round(a)
        a = rupiah_format(a, True)
        
        b = int(transaksi['transaksi']) / int(bulan)
        b = round(b)
        
        #bulan
        conn.execute('SELECT DATE_FORMAT(waktu_pesanan_selesai, "%b %y") bulan FROM penjualan WHERE status_pesanan = "Selesai" GROUP BY bulan ORDER BY waktu_pesanan_selesai ASC LIMIT 5 OFFSET 22')
        bulan = conn.fetchall()
        #jumlah transaksi/bulan dari 5 bulan terakhir
        conn.execute('SELECT COUNT(no_pesanan) transaksi FROM penjualan WHERE status_pesanan = "Selesai" GROUP BY DATE_FORMAT(waktu_pesanan_selesai, "%b %y") ORDER BY waktu_pesanan_selesai ASC LIMIT 5 OFFSET 22')
        total_transaksi = conn.fetchall()
        
        #top pelanggan
        conn.execute('SELECT nama_penerima, channel, COUNT(no_pesanan) frekuensi, CONCAT("Rp. ", FORMAT((SUM(total_harga)), 0, "de_DE")) totharga FROM `penjualan` WHERE status_pesanan = "Selesai" GROUP BY username_pembeli ORDER BY SUM(total_harga) DESC LIMIT 10')
        top_pelanggan = conn.fetchall()
        
        #top 5 provinsi
        conn.execute('SELECT provinsi, COUNT(no_pesanan) AS jumlah FROM `penjualan` WHERE status_pesanan = "Selesai" GROUP BY provinsi ORDER BY jumlah DESC LIMIT 5')
        propinsi = conn.fetchall()
        
        #jumlah tiap channel
        conn.execute('SELECT channel, COUNT(no_pesanan) jumlah FROM `penjualan` WHERE status_pesanan = "Selesai" GROUP BY channel')
        ch = conn.fetchall()
        
        #total tiap channel
        conn.execute('SELECT SUM(total_harga) harga FROM `penjualan` WHERE status_pesanan = "Selesai" AND channel = "shopee" GROUP BY channel')
        sho = conn.fetchone()
        sho = int(sho['harga'])
        conn.execute('SELECT SUM(total_harga) harga FROM `penjualan` WHERE status_pesanan = "Selesai" AND channel = "whatsapp" GROUP BY channel')
        wa = conn.fetchone()
        wa = int(wa['harga'])
        
        #total whatsapp 5 bulan
        conn.execute('SELECT SUM(total_harga) jumlah FROM `penjualan` WHERE status_pesanan = "Selesai" AND channel = "whatsapp" GROUP BY DATE_FORMAT(waktu_pesanan_selesai, "%b %y") ORDER BY waktu_pesanan_selesai DESC LIMIT 5')
        tot_wa = conn.fetchall()
        conn.execute('SELECT SUM(total_harga) jumlah FROM `penjualan` WHERE status_pesanan = "Selesai" AND channel = "shopee" GROUP BY DATE_FORMAT(waktu_pesanan_selesai, "%b %y") ORDER BY waktu_pesanan_selesai DESC LIMIT 5')
        tot_sho = conn.fetchall()
        
        #total whatsapp 5 bulan
        conn.execute('SELECT COUNT(DISTINCT no_pesanan) jumlah FROM `penjualan` WHERE status_pesanan = "Selesai" AND channel = "whatsapp" GROUP BY DATE_FORMAT(waktu_pesanan_selesai, "%b %y") ORDER BY waktu_pesanan_selesai DESC LIMIT 5')
        jum_wa = conn.fetchall()
        conn.execute('SELECT COUNT(no_pesanan) jumlah FROM `penjualan` WHERE status_pesanan = "Selesai" AND channel = "shopee" GROUP BY DATE_FORMAT(waktu_pesanan_selesai, "%b %y") ORDER BY waktu_pesanan_selesai DESC LIMIT 5')
        jum_sho = conn.fetchall()
        
        #jasa pengiriman
        conn.execute('SELECT opsi_pengiriman, COUNT(no_pesanan) jumtot FROM `penjualan` WHERE status_pesanan = "Selesai" GROUP BY opsi_pengiriman ORDER BY jumtot DESC LIMIT 5')
        opsi = conn.fetchall()
        return render_template('user/transaksi.html', jum_wa=jum_wa, jum_sho=jum_sho, opsi=opsi, tot_wa=tot_wa, tot_sho=tot_sho, sho=sho, wa=wa, ch=ch, propinsi=propinsi, top_pelanggan=top_pelanggan, bulan=bulan, total_transaksi=total_transaksi, a=a, b=b, transaksi=transaksi, total=total, account=account)
    # session tidak ada
    return redirect(url_for('login'))

#dashboard pelanggan
@app.route('/pelanggan')
def pelanggan():
    conn = mysql.connection.cursor()
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()
        #jumlah pelanggan
        conn.execute('SELECT COUNT(DISTINCT username_pembeli) pelanggan FROM penjualan WHERE status_pesanan = "Selesai"')
        pelanggan = conn.fetchone()
        #jumlah transaksi
        conn.execute('SELECT COUNT(no_pesanan) pesanan FROM `penjualan` WHERE status_pesanan = "Selesai"')
        transaksi = conn.fetchone()
        #jumlah pelanggan shopee
        conn.execute('SELECT username_pembeli sho FROM penjualan WHERE status_pesanan = "Selesai" AND channel = "shopee" GROUP BY username_pembeli')
        conn.fetchall()
        jumsho = conn.rowcount
        conn.execute('SELECT username_pembeli wa FROM penjualan WHERE status_pesanan = "Selesai" AND channel = "whatsapp" GROUP BY username_pembeli')
        conn.fetchall()
        jumwa = conn.rowcount
        
        #top pelanggan pengeluaran
        conn.execute('SELECT nama_penerima, channel, CONCAT("Rp. ", FORMAT((SUM(total_harga)), 0, "de_DE")) totharga, COUNT(no_pesanan) frekuensi, nama_produk FROM `penjualan` WHERE status_pesanan = "Selesai" GROUP BY username_pembeli ORDER BY SUM(total_harga) DESC LIMIT 10')
        top_pelanggan = conn.fetchall()
        
        #top pelanggan frekuensi
        conn.execute('SELECT nama_penerima, channel, COUNT(no_pesanan) frekuensi FROM `penjualan` WHERE status_pesanan = "Selesai" GROUP BY username_pembeli ORDER BY frekuensi DESC LIMIT 10')
        top_pelanggan1 = conn.fetchall()
        
        #bulan
        conn.execute('SELECT DATE_FORMAT(waktu_pesanan_selesai, "%b %y") bulan FROM penjualan WHERE status_pesanan = "Selesai" GROUP BY bulan ORDER BY waktu_pesanan_selesai ASC LIMIT 5 OFFSET 22')
        bulan = conn.fetchall()
        #total whatsapp 5 bulan
        conn.execute('SELECT COUNT(no_pesanan) jumlah FROM `penjualan` WHERE status_pesanan = "Selesai" AND channel = "whatsapp" GROUP BY DATE_FORMAT(waktu_pesanan_selesai, "%b %y") ORDER BY waktu_pesanan_selesai DESC LIMIT 5')
        jum_wa = conn.fetchall()
        conn.execute('SELECT COUNT(no_pesanan) jumlah FROM `penjualan` WHERE status_pesanan = "Selesai" AND channel = "shopee" GROUP BY DATE_FORMAT(waktu_pesanan_selesai, "%b %y") ORDER BY waktu_pesanan_selesai DESC LIMIT 5')
        jum_sho = conn.fetchall()
        
        #jumlah pelanggan provinsi
        conn.execute('SELECT COUNT(DISTINCT(username_pembeli)) jum, provinsi FROM penjualan GROUP BY provinsi ORDER BY jum DESC LIMIT 5')
        pro = conn.fetchall()
        
        #jumlah pelanggan 5 bln
        conn.execute('SELECT COUNT(DISTINCT username_pembeli) AS pelanggan FROM penjualan WHERE status_pesanan = "Selesai" AND channel="shopee" GROUP BY DATE_FORMAT(waktu_pesanan_selesai, "%b %y") ORDER BY waktu_pesanan_selesai DESC LIMIT 5')
        sho = conn.fetchall()
        conn.execute('SELECT COUNT(DISTINCT username_pembeli) AS pelanggan FROM penjualan WHERE status_pesanan = "Selesai" AND channel="whatsapp" GROUP BY DATE_FORMAT(waktu_pesanan_selesai, "%b %y") ORDER BY waktu_pesanan_selesai DESC LIMIT 5')
        wa = conn.fetchall()
        return render_template('user/pelanggan.html', bulan=bulan, wa=wa, sho=sho, pro=pro, top_pelanggan1=top_pelanggan1, top_pelanggan=top_pelanggan, jumsho=jumsho, jumwa=jumwa, transaksi=transaksi, pelanggan=pelanggan, account=account)
    # session tidak ada
    return redirect(url_for('login'))

#dashboard produk
@app.route('/produk')
def produk():
    conn = mysql.connection.cursor()
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()
        
        #jumlah barang
        conn.execute('SELECT COUNT(DISTINCT nama_produk) produk FROM `penjualan`')
        barang = conn.fetchone()
        
        #produk terjual/bulan
        conn.execute('SELECT SUM(jumlah) jum FROM penjualan WHERE status_pesanan = "Selesai"')
        produk = conn.fetchone()
        conn.execute('SELECT COUNT(no_pesanan) AS bulan FROM penjualan WHERE status_pesanan = "Selesai" GROUP BY DATE_FORMAT(waktu_pesanan_selesai, "%b %y")')
        conn.fetchall()
        bulan = conn.rowcount
        #pendapatan/bulan
        a = int(produk['jum']) / int(bulan)
        a = round(a)
        
        #top produk
        conn.execute('SELECT nama_produk AS produk, SUM(jumlah) AS jumlah FROM `penjualan` WHERE status_pesanan = "Selesai" AND nama_produk NOT IN ("Pesanan", "TITIPAN") GROUP BY nama_produk ORDER BY jumlah DESC LIMIT 10')
        top_produk = conn.fetchall()
        
        #produk paling hot
        conn.execute('SELECT nama_produk, CONCAT("Rp. ", FORMAT(total_harga, 0, "de_DE")) total_harga FROM `penjualan` ORDER BY harga DESC LIMIT 10')
        hot = conn.fetchall()
        
        #bulan
        conn.execute('SELECT DATE_FORMAT(waktu_pesanan_selesai, "%b %y") bulan FROM penjualan WHERE status_pesanan = "Selesai" GROUP BY bulan ORDER BY waktu_pesanan_selesai ASC LIMIT 5 OFFSET 22')
        bulan = conn.fetchall()
        conn.execute('SELECT COUNT(jumlah) jumlah FROM `penjualan` WHERE channel = "whatsapp" GROUP BY DATE_FORMAT(waktu_pesanan_selesai, "%b %y") ORDER BY waktu_pesanan_selesai DESC LIMIT 5')
        wa = conn.fetchall()
        conn.execute('SELECT COUNT(jumlah) jumlah FROM `penjualan` WHERE channel = "shopee" GROUP BY DATE_FORMAT(waktu_pesanan_selesai, "%b %y") ORDER BY waktu_pesanan_selesai DESC LIMIT 5')
        sho = conn.fetchall()
        return render_template('user/produk.html', wa=wa, sho=sho, bulan=bulan, hot=hot, top_produk=top_produk, a=a, barang=barang, account=account)
    # session tidak ada
    return redirect(url_for('login'))

#logout
@app.route('/logout')
def logout():
   session.pop('loggedin', None)
   session.pop('id', None)
   session.pop('username', None)
   return redirect(url_for('login'))

#data barang
@app.route("/masterbarang")
def masterbarang():
    conn = mysql.connection.cursor()
    conn.execute('SELECT nama_produk AS nama, nama_variasi, CONCAT("Rp. ", FORMAT((harga), 0, "de_DE")) harga FROM penjualan GROUP BY nama_produk, nama_variasi')
    barang = conn.fetchall()
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()
        return render_template('user/databarang.html', menu='master', submenu='barang', barang=barang, account=account)
    # session tidak ada
    return redirect(url_for('login'))

#data pelanggan
@app.route("/masterpelanggan")
def masterpelanggan():
    conn = mysql.connection.cursor()
    conn.execute("SELECT nama_penerima AS nama, alamat_pengiriman AS alamat, no_telepon AS telepon FROM penjualan GROUP BY username_pembeli")
    pelanggan = conn.fetchall()
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()
        return render_template('user/datapelanggan.html', menu='master', submenu='pelanggan', pelanggan=pelanggan, account=account)
    # session tidak ada
    return redirect(url_for('login'))

##PENJUALAN
@app.route("/formpenjualan")
def formpenjualan():
    conn = mysql.connection.cursor()
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()
        return render_template('user/formpenjualan.html', menu='penjualan', submenu='formpenjualan', account=account)
    # session tidak ada
    return redirect(url_for('login'))

@app.route("/simpanformpenjualan", methods=["POST"])
def simpanformpenjualan():
    if request.form['username_pembeli'] !="" and request.form['nama_pembeli'] !="" and request.form.getlist("produk[]") !="" and request.form.getlist("harga[]") !="" and request.form.getlist("jumlah[]") !="" and request.form['no_telepon'] !="" and request.form['alamat'] !="" and request.form['kota_kab'] !="" and request.form['provinsi'] !="" and request.form['waktu_selesai'] !="" and request.form.getlist("variasi[]") !="" and request.form['ongkir'] !="" and request.form['ekspedisi']!="":
        username_pembeli = request.form['username_pembeli']
        nama_pembeli = request.form['nama_pembeli']
        produk = request.form.getlist("produk[]")
        harga = request.form.getlist("harga[]")
        jumlah = request.form.getlist("jumlah[]")
        no_telepon = request.form['no_telepon']
        alamat = request.form['alamat']
        kota_kab = request.form['kota_kab']
        provinsi = request.form['provinsi']
        waktu_selesai = request.form['waktu_selesai']
        variasi = request.form.getlist("variasi[]")
        ongkir = request.form['ongkir']
        ekspedisi = request.form['ekspedisi']
        channel = "whatsapp"
        status_pesanan = "Selesai"
        #now = datetime.datetime(waktu_selesai)
        year, month, day = map(int, waktu_selesai.split('-'))
        now = str(year) + str(month)+ str(day)
        no_pesanan = "WHTSP"

        conn = mysql.connection.cursor()
        #format_tanggal = "%Y-%m-%d"
        #waktu_pesanan_selesai = datetime.strptime(waktu_selesai, format_tanggal)
        #waktu_pesanan_selesai = str(waktu_pesanan_selesai)
        conn.execute('SELECT COUNT(no_pesanan) transaksi FROM penjualan WHERE channel = "whatsapp" AND waktu_pesanan_selesai = %s', (waktu_selesai,))
        a = conn.fetchone()
        a = int(a['transaksi']) + 1
        a = str(a)
        no_pesanan = no_pesanan + now + a
        #no_pesanan = "WHTSP20190513" +a
        for index in range(len(produk)):
            total1 = int(harga[index]) * int(jumlah[index])
            conn.execute("INSERT INTO `penjualan`(`no_pesanan`,`status_pesanan`,`nama_produk`, `harga`, `jumlah`, `total_harga`,`username_pembeli`, `nama_penerima`, `no_telepon`, `alamat_pengiriman`, `kota_kab`, `provinsi`, `waktu_pesanan_selesai`, `channel`, `nama_variasi`, `ongkos_kirim`, `opsi_pengiriman`) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", (no_pesanan,status_pesanan, produk[index], harga[index], jumlah[index], total1, username_pembeli, nama_pembeli, no_telepon, alamat, kota_kab, provinsi, waktu_selesai, channel, variasi[index], ongkir, ekspedisi))
            mysql.connection.commit()
        flash('Berhasil masuk')
        conn.close()
        return redirect(url_for('datapenjualan'))
    else:
        flash('Ada data yang belum terisi')
        return redirect(url_for('formpenjualan'))
    
@app.route("/edit_penjualan/<string:id>", methods = ['GET'])
def edit_penjualan(id):
    conn = mysql.connection.cursor()
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()
        conn.execute("SELECT DISTINCT `no_pesanan`, `username_pembeli`, `nama_penerima`, `no_telepon`, `alamat_pengiriman`, `kota_kab`, `provinsi`, `waktu_pesanan_selesai`, `ongkos_kirim`, `opsi_pengiriman` FROM penjualan WHERE no_pesanan = %s", (id,))
        data = conn.fetchall()
        conn.execute("SELECT  `nama_produk`, `nama_variasi`, `harga`, `jumlah` FROM penjualan WHERE no_pesanan = %s", (id,))
        data1 = conn.fetchall()
        conn.close()
        return render_template('user/edit_penjualan.html', data = data, data1 = data1, menu='penjualan', submenu='formpenjualan', account=account)
    # session tidak ada
    return redirect(url_for('login'))

@app.route("/simpaneditpenjualan",methods=["POST","GET"])
def simpaneditpenjualan():
    conn = mysql.connection.cursor()
    no_pesanan = request.form['no_pesanan']
    if request.form['waktu_selesai'] !="":
        waktu_selesai = request.form['waktu_selesai']
    else:
        conn.execute("SELECT waktu_pesanan_selesai FROM penjualan WHERE no_pesanan = %s", (no_pesanan,))
        tgl = conn.fetchone()
        waktu_selesai = tgl['waktu_pesanan_selesai']
    username_pembeli = request.form['username_pembeli']
    nama_pembeli = request.form['nama_pembeli']
    produk = request.form.getlist("produk[]")
    harga = request.form.getlist("harga[]")
    jumlah = request.form.getlist("jumlah[]")
    no_telepon = request.form['no_telepon']
    alamat = request.form['alamat']
    kota_kab = request.form['kota_kab']
    provinsi = request.form['provinsi']
    variasi = request.form.getlist("variasi[]")
    ongkir = request.form['ongkir']
    ekspedisi = request.form['ekspedisi']
    channel = "whatsapp"
    status_pesanan = "Selesai"
    conn.execute("DELETE FROM penjualan WHERE no_pesanan = %s", (no_pesanan,))
    for index in range(len(produk)):
        total1 = int(harga[index]) * int(jumlah[index])
        conn.execute("INSERT INTO `penjualan`(`no_pesanan`,`status_pesanan`,`nama_produk`, `harga`, `jumlah`, `total_harga`,`username_pembeli`, `nama_penerima`, `no_telepon`, `alamat_pengiriman`, `kota_kab`, `provinsi`, `waktu_pesanan_selesai`, `channel`, `nama_variasi`, `ongkos_kirim`, `opsi_pengiriman`) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", (no_pesanan,status_pesanan, produk[index], harga[index], jumlah[index], total1, username_pembeli, nama_pembeli, no_telepon, alamat, kota_kab, provinsi, waktu_selesai, channel, variasi[index], ongkir, ekspedisi))
        mysql.connection.commit()
    flash('Berhasil update')
    conn.close()
    return redirect(url_for('datapenjualan'))

@app.route("/formpenjualan_excel")
def formpenjualan_excel():
    conn = mysql.connection.cursor()
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()
        return render_template('user/formpenjualan_excel.html', menu='penjualan', submenu='formpenjualan', account=account)
    # session tidak ada
    return redirect(url_for('login'))

# upload excel
UPLOAD_FOLDER_EXCEL = 'static/excel'
app.config['UPLOAD_FOLDER_EXCEL'] = UPLOAD_FOLDER_EXCEL

ALLOWED_EXTENSIONS_EXCEL = set(['csv', 'xls', 'xlsx'])
  
def allowed_file_excel(filename):
 return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS_EXCEL

@app.route("/simpanformpenjualan_excel", methods=["POST","GET"])
def simpanformpenjualan_excel():
    conn = mysql.connection.cursor()
    if request.method == 'POST':
        file = request.files['file']
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER_EXCEL'], filename))
        namefile = "C:/Users/ASUS A407U/skripsi/static/excel/"+filename
        excel = xlrd.open_workbook(namefile)
        sheet = excel.sheet_by_index(0)
        jum_row = sheet.nrows
        for baris in range(jum_row):
            if baris == 0:
                continue
            else:
                # insert data
                sql = "INSERT INTO `penjualan`(`no_pesanan`, `status_pesanan`, `alasan_pembatalan`, `status_pembatalan`, `no_resi`, `opsi_pengiriman`, `antar_pengiriman`, `waktu_pengiriman_diatur`, `waktu_pesanan_dibuat`, `waktu_pembayaran_dilakukan`, `nama_produk`, `nama_variasi`, `harga`, `jumlah`, `total_harga`, `berat_produk`, `ongkos_kirim`, `username_pembeli`, `nama_penerima`, `no_telepon`, `alamat_pengiriman`, `kota_kab`, `provinsi`, `waktu_pesanan_selesai`, `channel`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, 'shopee')"
                val = tuple(sheet.row_values(baris))
                conn.execute(sql, val)
                mysql.connection.commit()
    flash('Berhasil masuk')
    conn.close()
    return redirect(url_for('datapenjualan'))

@app.route("/datapenjualan")
def datapenjualan():
    conn = mysql.connection.cursor()
    conn.execute("SELECT `no_pesanan`, `status_pesanan`, CONCAT('Rp. ', FORMAT(SUM((total_harga)), 0, 'de_DE')) AS harga, `nama_penerima`, `kota_kab`, `provinsi` FROM penjualan GROUP BY no_pesanan ORDER BY waktu_pesanan_selesai DESC")
    penjualan = conn.fetchall()
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()
        return render_template('user/datapenjualan.html', menu='penjualan', submenu='datapenjualan', penjualan=penjualan, account=account)
    # session tidak ada
    return redirect(url_for('login'))

@app.route('/view_penjualan/<id>', methods = ['POST', 'GET'])
def view_penjualan(id):
    conn = mysql.connection.cursor()
    
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()
        conn.execute("SELECT `no_pesanan`, `status_pesanan`, `alasan_pembatalan`, `status_pembatalan`, `no_resi`, `opsi_pengiriman`, `antar_pengiriman`, `waktu_pengiriman_diatur`, `waktu_pesanan_dibuat`, `waktu_pembayaran_dilakukan`, CONCAT('Rp. ', FORMAT(SUM(total_harga), 0, 'de_DE')) total_harga, CONCAT('Rp. ', FORMAT(ongkos_kirim, 0, 'de_DE')) ongkos_kirim, `username_pembeli`, `nama_penerima`, `no_telepon`, `alamat_pengiriman`, `kota_kab`, `provinsi`, `waktu_pesanan_selesai` FROM penjualan WHERE no_pesanan = %s GROUP BY no_pesanan", (id,))
        data = conn.fetchall()
        conn.execute("SELECT nama_produk, nama_variasi, CONCAT('Rp. ', FORMAT(harga, 0, 'de_DE')) harga, `jumlah`, CONCAT('Rp. ', FORMAT(total_harga, 0, 'de_DE')) total_harga FROM penjualan WHERE no_pesanan = %s", (id,))
        barang = conn.fetchall()
        conn.close()
        return render_template('user/view_penjualan.html', barang=barang, penjualan = data, menu='penjualan', submenu='datapenjualan', account=account)
    # session tidak ada
    return redirect(url_for('login'))

@app.route('/delete_penjualan/<string:id>', methods = ['GET'])
def delete_penjualan(id):
    conn = mysql.connection.cursor()
    conn.execute("DELETE FROM penjualan WHERE no_pesanan = %s", (id,))
    mysql.connection.commit()
    conn.close()
    flash('Berhasil dihapus')
    return redirect(url_for('datapenjualan'))

def inputNumber(message):
  while True:
    try:
       userInput = int(input(message))       
    except ValueError:
       print("Not an integer! Try again.")
       continue
    else:
       return userInput 
       break 

@app.route("/katalog",methods=["POST","GET"])
def homepage():
    conn = mysql.connection.cursor()
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()
        return render_template('user/page_catalogue.html', menu='katalog', submenu='entrikatalog', account=account)
    # session tidak ada
    return redirect(url_for('login'))

# upload gambar
UPLOAD_FOLDER = 'static/photos'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg'])
  
def allowed_file(filename):
 return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/display/<filename>')
def display(filename):
    #print('display_image filename: ' + filename)
    return redirect(url_for('static', filename='photos/' + filename), code=301)

@app.route("/upload",methods=["POST","GET"])
def upload():
    cursor = mysql.connection.cursor()
    cur = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    now = datetime.now()
    if request.method == 'POST':
        if request.form['nama_produk'] != "":
            if request.form['nama_variasi'] != "":
                if request.form['harga'] !="":
                    nama_produk = request.form['nama_produk']
                    nama_variasi = request.form['nama_variasi']
                    harga = request.form['harga']
                    file = request.files['file']
                    cur.execute('SELECT MAX(id_produk) max FROM produk')
                    max = cur.fetchone()
                    type = file.filename.rsplit('.', 1)[1].lower()
                    judul = max['max']
                    judul = int(judul) + 1
                    judul = str(judul) + "." + type
                    filename = judul 
                    file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
                    cur.execute("INSERT INTO `produk`(`nama_produk`, `nama_variasi`, `harga`, `foto`) VALUES (%s,%s,%s,%s)",[nama_produk, nama_variasi, harga, filename])
                    mysql.connection.commit()
                    cur.close()   
                    flash('Berhasil upload')
                    return redirect('/datakatalog')
                else:
                    flash('Gagal upload, Ada data yang belum terisi')
                    return redirect('/katalog')
            else:
                flash('Gagal upload, Ada data yang belum terisi')
                return redirect('/katalog')
        else:
            flash('Gagal upload, Ada data yang belum terisi')
            return redirect('/katalog')

@app.route("/datakatalog")
def datakatalog():
    conn = mysql.connection.cursor()
    conn.execute("SELECT `id_produk`, `nama_produk`, `nama_variasi`, CONCAT('Rp. ', FORMAT((harga), 0, 'de_DE')) harga, `foto`, `set` FROM `produk`")
    katalog = conn.fetchall()
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()
        return render_template('user/data_katalog.html', menu='katalog', submenu='datakatalog', katalog=katalog, account=account)
    # session tidak ada
    return redirect(url_for('login'))

@app.route('/set_katalog/<string:id>', methods = ['GET'])
def set_katalog(id):
    conn = mysql.connection.cursor()
    conn.execute("UPDATE `produk` SET `set`= 0 WHERE id_produk = %s", (id,))
    mysql.connection.commit()
    conn.close()
    flash('Berhasil disembunyikan')
    return redirect(url_for('datakatalog'))

@app.route('/set_katalog1/<string:id>', methods = ['GET'])
def set_katalog1(id):
    conn = mysql.connection.cursor()
    conn.execute("UPDATE `produk` SET `set`= 1 WHERE id_produk = %s", (id,))
    mysql.connection.commit()
    conn.close()
    flash('Berhasil ditampilkan')
    return redirect(url_for('datakatalog'))

@app.route('/delete_katalog/<string:id>', methods = ['GET'])
def delete_katalog(id):
    conn = mysql.connection.cursor()
    conn.execute("DELETE FROM `produk` WHERE id_produk = %s", (id,))
    mysql.connection.commit()
    conn.close()
    flash('Berhasil dihapus')
    return redirect(url_for('datakatalog'))

@app.route('/edit_katalog/<string:id>', methods = ['GET'])
def edit_katalog(id):
    conn = mysql.connection.cursor()
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()
        conn.execute("SELECT * FROM `produk` WHERE id_produk = %s", (id,))
        data = conn.fetchall()
        conn.close()
        return render_template('user/edit_katalog.html', data = data, menu='katalog', submenu='datakatalog', account=account)
    # session tidak ada
    return redirect(url_for('login'))

@app.route("/upload_edit_katalog",methods=["POST","GET"])
def upload_edit_katalog():
    cursor = mysql.connection.cursor()
    cur = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    nama_produk = request.form['nama_produk']
    nama_variasi = request.form['nama_variasi']
    harga = request.form['harga']
    file = request.files['file']
    id_produk = request.form['id_produk'] 
    if 'file' not in request.files:
        cur.execute("UPDATE `produk` SET `nama_produk`=%s,`nama_variasi`=%s,`harga`=%s WHERE id_produk=%s",[nama_produk, nama_variasi, harga, id_produk])
        mysql.connection.commit()
        cur.close()     
        flash('Berhasil diupdate')
    if file.filename == '':
        cur.execute("UPDATE `produk` SET `nama_produk`=%s,`nama_variasi`=%s,`harga`=%s WHERE id_produk=%s",[nama_produk, nama_variasi, harga, id_produk])
        mysql.connection.commit()
        cur.close()    
        flash('Berhasil diupdate')
    else:
        cur.execute('SELECT MAX(id_produk) max FROM produk')
        max = cur.fetchone()
        type = file.filename.rsplit('.', 1)[1].lower()
        judul = max['max']
        judul = int(judul) + 1
        judul = str(judul) + "." + type
        filename = judul 
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        cur.execute("UPDATE `produk` SET `nama_produk`=%s,`nama_variasi`=%s,`harga`=%s, `foto`=%s WHERE id_produk=%s",[nama_produk, nama_variasi, harga, filename, id_produk])
        mysql.connection.commit()
        cur.close()   
        flash('Berhasil diupdate')    
    return redirect('/datakatalog')

@app.route("/hasil_lrfm")
def hasil_lrfm():
    conn = mysql.connection.cursor()
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()
        conn.execute('SELECT username_pembeli AS pelanggan, TIMESTAMPDIFF(DAY,MIN(waktu_pesanan_selesai), "2021-06-30") AS length, TIMESTAMPDIFF(DAY,MAX(waktu_pesanan_selesai), "2021-06-30") AS recency, COUNT(no_pesanan) AS frequency, CONCAT("Rp. ", FORMAT((CONVERT(SUM(total_harga), INT)), 0, "de_DE")) AS monetary FROM penjualan WHERE status_pesanan = "Selesai" GROUP BY username_pembeli')
        df= pd.DataFrame(conn.fetchall(), columns=['pelanggan', 'length', 'recency', 'frequency', 'monetary'])
        df = tuple(df.itertuples(index=False, name=None))
        conn.close()
        return render_template('user/hasil_lrfm.html', df=df, menu='segmentasi', submenu='hasil_lrfm', account=account)
    # session tidak ada
    return redirect(url_for('login'))

@app.route("/hasil_segmentasi")
def hasil_segmentasi():
    conn = mysql.connection.cursor()
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()  
        #query LRFM
        conn.execute('SELECT username_pembeli AS pelanggan, TIMESTAMPDIFF(DAY,MIN(waktu_pesanan_selesai), "2021-06-30") AS length, TIMESTAMPDIFF(DAY,MAX(waktu_pesanan_selesai), "2021-06-30") AS recency, COUNT(no_pesanan) AS frequency, CONVERT(SUM(total_harga), INT) AS monetary FROM penjualan WHERE status_pesanan = "Selesai" GROUP BY username_pembeli ORDER BY username_pembeli')
        df= pd.DataFrame(conn.fetchall(), columns=['pelanggan', 'length', 'recency', 'frequency', 'monetary'])
        
        #normalisasi
        def length_nor(row):
            x = ((row['length']-min(df['length']))/(max(df['length'])-min(df['length'])))*(1-0)+0
            return x
        def recency_nor(row):
            y = ((row['recency']-min(df['recency']))/(max(df['recency'])-min(df['recency'])))*(1-0)+0
            return y
        def frequency_nor(row):
            z = ((row['frequency']-min(df['frequency']))/(max(df['frequency'])-min(df['frequency'])))*(1-0)+0
            return z
        def monetary_nor(row):
            w = ((row['monetary']-min(df['monetary']))/(max(df['monetary'])-min(df['monetary'])))*(1-0)+0
            return w
        
        #normalisasi ke df
        df['length_nor'] = df.apply(lambda row: length_nor(row), axis=1)
        df['recency_nor'] = df.apply(lambda row: recency_nor(row), axis=1)
        df['frequency_nor'] = df.apply(lambda row: frequency_nor(row), axis=1)
        df['monetary_nor'] = df.apply(lambda row: monetary_nor(row), axis=1)
        
        #drop LRFM asli
        df = df.drop(['length'], axis=1)
        df = df.drop(['recency'], axis=1)
        df = df.drop(['frequency'], axis=1)
        df = df.drop(['monetary'], axis=1)
        
        #clustering
        z = df.iloc[:, [1,2,3,4]].values
        selected_cols = ["pelanggan", "length_nor", "recency_nor", "frequency_nor", "monetary_nor"]
        cluster_data = df.loc[:,selected_cols]
        kmeans_s = KMeans(n_clusters=4, random_state=0).fit(z)
        label = pd.DataFrame(kmeans_s.labels_)
        clustered_data = cluster_data.assign(cluster=label)
        clustered_data['length_nor'] = clustered_data['length_nor'].round(6)
        clustered_data['recency_nor'] = clustered_data['recency_nor'].round(6)
        clustered_data['frequency_nor'] = clustered_data['frequency_nor'].round(6)
        clustered_data['monetary_nor'] = clustered_data['monetary_nor'].round(6)
        
        #jumlah data tiap cluster
        cluster_0 = clustered_data[clustered_data['cluster']==0]['pelanggan'].count()
        cluster_1 = clustered_data[clustered_data['cluster']==1]['pelanggan'].count()
        cluster_2 = clustered_data[clustered_data['cluster']==2]['pelanggan'].count()
        cluster_3 = clustered_data[clustered_data['cluster']==3]['pelanggan'].count()
        
        #centroid
        grouped_km = clustered_data.groupby(['cluster']).mean().round(6)
        grouped_km2 = clustered_data.groupby(['cluster']).mean().round(6).reset_index()
        grouped_km2['cluster'] = grouped_km2['cluster'].map(str)
        grouped_km3 = clustered_data.groupby(['cluster']).mean().round(3).reset_index()
        grouped_km3['cluster'] = grouped_km3['cluster'].map(str)
        def length_nor3(row):
            if row['length_nor'] <= 0.200:
                x = "Kecil"
            elif (row['length_nor'] > 0.200 and row['length_nor'] <= 0.500):
                x = "Sedang"
            elif (row['length_nor'] > 0.500):
                x = "Besar"
            return x
        def recency_nor3(row):
            if row['recency_nor'] <= 0.200:
                y = "Kecil"
            elif (row['recency_nor'] > 0.200 and row['recency_nor'] <= 0.600):
                y = "Sedang"
            elif (row['recency_nor'] > 0.600):
                y = "Besar"
            return y
        def frequency_nor3(row):
            if row['frequency_nor'] >= 0.100:
                z = "Besar"
            elif (row['frequency_nor'] > 0.003 and row['frequency_nor'] <= 0.008):
                z = "Sedang"
            elif (row['frequency_nor'] <= 0.003):
                z = "Kecil"
            return z
        def monetary_nor3(row):
            if row['monetary_nor'] >= 0.100:
                w = "Besar"
            elif (row['monetary_nor'] > 0.010 and row['monetary_nor'] <= 0.020):
                w = "Sedang"
            elif (row['monetary_nor'] <= 0.010):
                w = "Kecil"
            return w
        print (grouped_km3)
        grouped_km3['length_nor'] = grouped_km3.apply(lambda row: length_nor3(row), axis=1)
        grouped_km3['recency_nor'] = grouped_km3.apply(lambda row: recency_nor3(row), axis=1)
        grouped_km3['frequency_nor'] = grouped_km3.apply(lambda row: frequency_nor3(row), axis=1)
        grouped_km3['monetary_nor'] = grouped_km3.apply(lambda row: monetary_nor3(row), axis=1)
        grouped_km3 = tuple(grouped_km3.itertuples(index=False, name=None))
        grouped_km2 = tuple(grouped_km2.itertuples(index=False, name=None))
        data_cluster = [cluster_0, cluster_1, cluster_2, cluster_3]
        
        
        clustered_data = tuple(clustered_data.itertuples(index=False, name=None))
        conn.close()
        return render_template('user/hasil_segmentasi.html', grouped_km3=grouped_km3, grouped_km2=grouped_km2, label=label, data_cluster=data_cluster, clustered_data=clustered_data, menu='segmentasi', submenu='hasil_segmentasi', account=account)
    # session tidak ada
    return redirect(url_for('login'))

@app.route("/datacluster",methods=["POST","GET"])
def datacluster():
    conn = mysql.connection.cursor()
    # cek session
    if 'loggedin' in session:
        # informasi user dari session
        conn.execute('SELECT * FROM accounts WHERE id = %s', [session['id']])
        account = conn.fetchone()  
        #query LRFM
        conn.execute('SELECT username_pembeli AS pelanggan, TIMESTAMPDIFF(DAY,MIN(waktu_pesanan_selesai), "2021-06-30") AS length, TIMESTAMPDIFF(DAY,MAX(waktu_pesanan_selesai), "2021-06-30") AS recency, COUNT(no_pesanan) AS frequency, CONVERT(SUM(total_harga), INT) AS monetary FROM penjualan WHERE status_pesanan = "Selesai" GROUP BY username_pembeli ORDER BY username_pembeli')
        df= pd.DataFrame(conn.fetchall(), columns=['pelanggan', 'length', 'recency', 'frequency', 'monetary'])
        
        #normalisasi
        def length_nor(row):
            x = ((row['length']-min(df['length']))/(max(df['length'])-min(df['length'])))*(1-0)+0
            return x
        def recency_nor(row):
            y = ((row['recency']-min(df['recency']))/(max(df['recency'])-min(df['recency'])))*(1-0)+0
            return y
        def frequency_nor(row):
            z = ((row['frequency']-min(df['frequency']))/(max(df['frequency'])-min(df['frequency'])))*(1-0)+0
            return z
        def monetary_nor(row):
            w = ((row['monetary']-min(df['monetary']))/(max(df['monetary'])-min(df['monetary'])))*(1-0)+0
            return w
        
        #normalisasi ke df
        df['length_nor'] = df.apply(lambda row: length_nor(row), axis=1)
        df['recency_nor'] = df.apply(lambda row: recency_nor(row), axis=1)
        df['frequency_nor'] = df.apply(lambda row: frequency_nor(row), axis=1)
        df['monetary_nor'] = df.apply(lambda row: monetary_nor(row), axis=1)
        
        #drop LRFM asli
        df = df.drop(['length'], axis=1)
        df = df.drop(['recency'], axis=1)
        df = df.drop(['frequency'], axis=1)
        df = df.drop(['monetary'], axis=1)
        
        #clustering
        z = df.iloc[:, [1,2,3,4]].values
        selected_cols = ["pelanggan", "length_nor", "recency_nor", "frequency_nor", "monetary_nor"]
        cluster_data = df.loc[:,selected_cols]
        kmeans_s = KMeans(n_clusters=4, random_state=0).fit(z)
        label = pd.DataFrame(kmeans_s.labels_)
        clustered_data = cluster_data.assign(cluster=label)
        clustered_data['length_nor'] = clustered_data['length_nor'].round(3)
        clustered_data['recency_nor'] = clustered_data['recency_nor'].round(3)
        clustered_data['frequency_nor'] = clustered_data['frequency_nor'].round(3)
        clustered_data['monetary_nor'] = clustered_data['monetary_nor'].round(3)
        clustered_data2 = pd.DataFrame(clustered_data['pelanggan'])
        clustered_data2.insert(value = clustered_data['cluster'], loc=1, column='cluster')
        
        conn.execute('SELECT nama_penerima AS nama FROM penjualan WHERE status_pesanan = "Selesai" GROUP BY username_pembeli ORDER BY username_pembeli')
        nama= pd.DataFrame(conn.fetchall(), columns=['nama'])
        clustered_data2.insert(value = nama.nama, loc=2, column='nama')
        conn.execute('SELECT no_telepon AS telepon FROM penjualan WHERE status_pesanan = "Selesai" GROUP BY username_pembeli ORDER BY username_pembeli')
        nama= pd.DataFrame(conn.fetchall(), columns=['telepon'])
        clustered_data2.insert(value = nama.telepon, loc=3, column='telepon')
        conn.execute('SELECT alamat_pengiriman AS alamat FROM penjualan WHERE status_pesanan = "Selesai" GROUP BY username_pembeli ORDER BY username_pembeli')
        nama= pd.DataFrame(conn.fetchall(), columns=['alamat'])
        clustered_data2.insert(value = nama.alamat, loc=4, column='alamat')
        if request.method == 'POST':
            cluster = request.form['c']
            if cluster == "c":
                clustered_data2 = tuple(clustered_data2.itertuples(index=False, name=None))
            elif cluster == "0":
                clustered_data2 = clustered_data2[clustered_data2['cluster']==0]
                clustered_data2 = tuple(clustered_data2.itertuples(index=False, name=None))
            elif cluster == "1":
                clustered_data2 = clustered_data2[clustered_data2['cluster']==1]
                clustered_data2 = tuple(clustered_data2.itertuples(index=False, name=None))
            elif cluster == "2":
                clustered_data2 = clustered_data2[clustered_data2['cluster']==2]
                clustered_data2 = tuple(clustered_data2.itertuples(index=False, name=None))
            elif cluster == "3":
                clustered_data2 = clustered_data2[clustered_data2['cluster']==3]
                clustered_data2 = tuple(clustered_data2.itertuples(index=False, name=None))
        else:
            clustered_data2 = tuple(clustered_data2.itertuples(index=False, name=None))
        conn.close()
        return render_template('user/datacluster.html', clustered_data2=clustered_data2, menu='segmentasi', submenu='datacluster', account=account)
    # session tidak ada
    return redirect(url_for('login'))

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Dec/2021 19:50:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2021 19:50:51] "GET /static/home/css/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2021 19:50:51] "GET /static/home/2/css/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2021 19:50:52] "GET /static/home/js/scripts.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2021 19:50:52] "GET /static/home/2/js/scripts.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2021 19:50:52] "GET /static/img/wa.png HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2021 19:50:52] "GET /static/home/2/assets/img/2.jpg HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2021 19:50:52] "GET /static/img/header.jpg HTTP/1.1" 200 -
[2021-12-26 19:51:28,418] ERROR in app: Exception on /home [GET]
Traceback (most recent call last):
  File "C:\Users\ASUS A407U\Anaconda3\lib\site-packages\flask\app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\ASUS A407U\Anaconda3\lib\site-p